In [ ]:
'''数据分析：删除缺失值、相关性分析、数据划分
特征处理：缺失值填充、编码、异常值处理、归一化、去重复值
模型训练：模型选择、调参、错误分析、训练集测试集评价
'''

# 数据分析

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('US-pumpkins.csv') 
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City Name        1757 non-null   object 
 1   Type             45 non-null     object 
 2   Package          1757 non-null   object 
 3   Variety          1752 non-null   object 
 4   Sub Variety      296 non-null    object 
 5   Grade            0 non-null      float64
 6   Date             1757 non-null   object 
 7   Low Price        1757 non-null   float64
 8   High Price       1757 non-null   float64
 9   Mostly Low       1654 non-null   float64
 10  Mostly High      1654 non-null   float64
 11  Origin           1754 non-null   object 
 12  Origin District  131 non-null    object 
 13  Item Size        1478 non-null   object 
 14  Color            1141 non-null   object 
 15  Environment      0 non-null      float64
 16  Unit of Sale     162 non-null    object 
 17  Quality       

,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,Mostly Low,...,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode,Unnamed: 24,Unnamed: 25
0,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,4/29/17,270.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
1,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,5/6/17,270.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
2,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.0,160.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
3,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.0,160.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
4,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,11/5/16,90.0,100.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN


In [6]:
# 设置缺失率阈值
threshold = 0.75

# 计算每列的缺失率
missing_ratio = df.isnull().mean()

print(missing_ratio)

City Name          0.000000
Type               0.974388
Package            0.000000
Variety            0.002846
Sub Variety        0.831531
Grade              1.000000
Date               0.000000
Low Price          0.000000
High Price         0.000000
Mostly Low         0.058623
Mostly High        0.058623
Origin             0.001707
Origin District    0.925441
Item Size          0.158793
Color              0.350598
Environment        1.000000
Unit of Sale       0.907797
Quality            1.000000
Condition          1.000000
Appearance         1.000000
Storage            1.000000
Crop               1.000000
Repack             0.000000
Trans Mode         1.000000
Unnamed: 24        1.000000
Unnamed: 25        0.941377
dtype: float64


In [7]:
# 筛选出缺失率低于阈值的列
cols_to_keep = missing_ratio[missing_ratio < threshold].index
print(cols_to_keep)

Index(['City Name', 'Package', 'Variety', 'Date', 'Low Price', 'High Price',
       'Mostly Low', 'Mostly High', 'Origin', 'Item Size', 'Color', 'Repack'],
      dtype='object')


In [8]:
cols_to_drop = missing_ratio[missing_ratio > threshold].index
print(cols_to_drop)

Index(['Type', 'Sub Variety', 'Grade', 'Origin District', 'Environment',
       'Unit of Sale', 'Quality', 'Condition', 'Appearance', 'Storage', 'Crop',
       'Trans Mode', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')


In [9]:
df = df.drop(cols_to_drop, axis=1)

In [10]:
df.shape

(1757, 12)

In [12]:
# 数值型列与 High Price 的相关性
numeric_cols = df.select_dtypes(include=[np.number]).columns
if 'High Price' in numeric_cols:
    high_price_corr = df[numeric_cols].corr()['High Price'].sort_values(ascending=False)
    print("Correlation with 'High Price' (numeric features):")
    print(high_price_corr)
else:
    print("'High Price' not found in numeric columns.")

Correlation with 'High Price' (numeric features):
High Price     1.000000
Mostly High    0.996174
Mostly Low     0.989539
Low Price      0.987353
Name: High Price, dtype: float64


In [13]:
# 非数值型列
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns

for col in non_numeric_cols:
    if 'High Price' in df.columns:
        grouped = df.groupby(col)['High Price'].mean().sort_values(ascending=False)
        print(f"\nAverage 'High Price' by '{col}':")
        print(grouped.head(10))  # 显示前10个类别


Average 'High Price' by 'City Name':
City Name
BOSTON          187.235795
MIAMI           170.000000
ATLANTA         153.416667
DALLAS          149.802920
COLUMBIA        141.790875
NEW YORK        116.482143
CHICAGO         115.691532
DETROIT         109.204545
BALTIMORE       109.055556
PHILADELPHIA    103.754386
Name: High Price, dtype: float64

Average 'High Price' by 'Package':
Package
36 inch bins      182.769306
24 inch bins      177.626842
bins              172.307692
each              103.102941
bushel baskets     49.100000
40 lb cartons      33.421053
50 lb cartons      30.400000
35 lb cartons      28.535714
50 lb sacks        27.090909
bushel cartons     24.027027
Name: High Price, dtype: float64

Average 'High Price' by 'Variety':
Variety
BLUE TYPE                   231.315789
KNUCKLE HEAD                199.500000
FAIRYTALE                   197.753333
CINDERELLA                  178.153086
MIXED HEIRLOOM VARIETIES    163.070175
HOWDEN TYPE                 157.684041
HOWD

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('US-pumpkins.csv') 
df.info()
df.head()

# 设置缺失率阈值
threshold = 0.75

# 计算每列的缺失率
missing_ratio = df.isnull().mean()

print(missing_ratio)

# 筛选出缺失率低于阈值的列
cols_to_keep = missing_ratio[missing_ratio < threshold].index
print(cols_to_keep)

cols_to_drop = missing_ratio[missing_ratio > threshold].index
print(cols_to_drop)

df = df.drop(cols_to_drop, axis=1)